In [0]:
import requests
import time, pytz, datetime
import csv
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/data/Kraken'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/data/Kraken/ltcusd


In [0]:
def _timestamp_to_datetime(timestamp):
  return datetime.datetime.fromtimestamp(timestamp)

def _datetime_to_str(dt,form):
  """
    dt: datetime obj
    form: format to return
  """
  return datetime.datetime.strftime(dt,form)

In [0]:
def _update_data(resp_json,symbol):
  """
    resp: the requests object
    symbol: str
  """
  last = resp_json['result']['last']
  last_time = resp_json['result'][symbol][-1][2]
  data = resp_json['result'][symbol]
  return(last, last_time, data)

In [0]:
def _write_file(last, bigdata, symbol):
  from_date = _datetime_to_str(_timestamp_to_datetime(float(bigdata[0][2])),'%Y-%m-%d %H:%M:%S')
  to_date = _datetime_to_str(_timestamp_to_datetime(float(bigdata[-1][2])),'%Y-%m-%d %H:%M:%S')
  file_name = 'Kraken_'+symbol+'_'+from_date+'_'+to_date+'_'+str(last)+'.csv'
  with open(file_name, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(bigdata)
  print('Generated '+file_name)

In [0]:
# Init values
symbol = "XLTCZUSD"
resp = requests.get('https://api.kraken.com/0/public/Trades?pair='+symbol+'&since=1524470967426846318')
json_resp = resp.json()

bigdata = json_resp['result'][symbol]
first_time = json_resp['result'][symbol][0][2]
last_time = json_resp['result'][symbol][-1][2]
last = json_resp['result']['last']

n = 1000000 # split file every n = 1,000,000 entries
file_count = 0
total_curr_len = 1
end_time = '2020-01-01'
end_time = datetime.datetime.timestamp(datetime.datetime.strptime(end_time,'%Y-%m-%d'))

In [0]:
while float(last_time) < end_time:

  while True:
    try:
      resp = requests.get('https://api.kraken.com/0/public/Trades?pair='+symbol+'&since=' + str(last))
      if resp.status_code == 200:
        resp_json = resp.json()
        last, last_time, data = _update_data(resp_json,symbol)
        bigdata += data
        total_curr_len += 1
        
        if (total_curr_len == 1000):
          _write_file(last,bigdata,symbol)
          bigdata = []
          total_curr_len = 0

      else:

        print(resp.status_code)
        break
      time.sleep(5)
    except Exception as e:
      print(e)
      print(resp_json)
      time.sleep(30)
    else:
      break

_write_file(last,bigdata,symbol)

Generated Kraken_XLTCZUSD_2018-04-23 08:10:53_2019-07-17 02:11:42_1563329502230952685.csv
Generated Kraken_XLTCZUSD_2019-07-17 02:11:44_2020-01-02 07:46:57_1577951217828517304.csv
